In [57]:
import math
import yaml
import torch
from collections import defaultdict

In [2]:
import sys
import importlib.util

solver_dir = "/home/ubuntu/SML/solver"

sys.path.append(solver_dir)

file_path = "/home/ubuntu/SML/solver/solver.py"
spec = importlib.util.spec_from_file_location("solver", file_path)
solver = importlib.util.module_from_spec(spec)
spec.loader.exec_module(solver)

In [3]:
E = 113.8e9
nu = 0.342
rho = 4.47e-3

device = "cuda:2"
dtype = torch.float32

In [4]:
i = 0

with open(f'/data/SimJEB/boundary/{i}.yaml') as file:
    boundary = yaml.safe_load(file)
    rbe2 = torch.cat([torch.tensor(rbe2['slaves']) for rbe2 in boundary['rbe2']]).to(device)
    rbe3 = torch.cat([torch.tensor(rbe3['slaves']) for rbe3 in boundary['rbe3']]).to(device)

coords, elements = solver.vtk_loader_to_torch(f'/data/SimJEB/vtk/{i}.vtk', 'c3d4')
coords = coords.to(device)
elements = elements.to(device)

In [5]:
print(rbe2.shape, rbe3.shape, coords.shape, elements.shape)

torch.Size([589]) torch.Size([753]) torch.Size([112873, 3]) torch.Size([570111, 4])


In [6]:
K = solver.compute_K_matrix(coords, elements, 'c3d4', E, nu, device=device, dtype=dtype)
print(K.shape)
print(f"Memory allocated on GPU for element wise K: {torch.cuda.memory_allocated(K.device) / 1024**3:.4f} GB")

torch.Size([570111, 12, 12])
Memory allocated on GPU for element wise K: 0.3341 GB


In [7]:
M = K.shape[0]

node_indices = elements.unsqueeze(-1).repeat(1, 1, 3)
dof_indices = node_indices * 3 + torch.arange(3, device=device).view(1, 1, 3)
dof_indices = dof_indices.view(M, -1)

row_idx = dof_indices.unsqueeze(2).repeat(1, 1, 12).view(-1)
col_idx = dof_indices.unsqueeze(1).repeat(1, 12, 1).view(-1)
values = K.view(-1)

N_nodes = elements.max().item() + 1
N_dof = N_nodes * 3

K_sparse = torch.sparse_coo_tensor(
    indices=torch.stack([row_idx, col_idx]),
    values=values,
    size=(N_dof, N_dof),
    device=device
)

print(K_sparse.shape)
print(f"Memory allocated on GPU for sparse K: {torch.cuda.memory_allocated(K_sparse.device) / 1024**3:.4f} GB")

torch.Size([338619, 338619])
Memory allocated on GPU for sparse K: 2.8827 GB


In [8]:
def compute_subdivisions(matrix_size, gpu_memory_gb):
    bytes_per_float = 4  # float32
    max_bytes = gpu_memory_gb * (1024 ** 3)
    max_elements = max_bytes // bytes_per_float
    max_dim = int(math.floor(math.sqrt(max_elements)))
    subdivisions = math.ceil(matrix_size / max_dim)
    return subdivisions

print(compute_subdivisions(K_sparse.shape[0], 10))

7


In [9]:
edge = solver.identify_tetrahedral_shared_faces(elements, device=device)
edge = torch.cat([edge[:,0,0].unsqueeze(0), edge[:,1,0].unsqueeze(0)], dim=0)
print(edge.shape)
print(f"Memory allocated on GPU for edge: {torch.cuda.memory_allocated(edge.device) / 1024**3:.4f} GB")

torch.Size([2, 1097840])
Memory allocated on GPU for edge: 2.8991 GB


In [10]:
def build_adjacency_matrix(edge, num_elements, device):
    edge = edge.to(device)
    indices = torch.cat([edge, edge[[1, 0]]], dim=1)
    values = torch.ones(indices.shape[1], device=device)
    adj = torch.sparse_coo_tensor(indices, values, size=(num_elements, num_elements), device=device)
    return adj.coalesce()

def pick_distant_seeds(adj, n_parts):
    N = adj.shape[0]
    device = adj.device
    seeds = [torch.randint(0, N, (1,), device=device).item()]
    selected_mask = torch.zeros(N, dtype=torch.bool, device=device)
    selected_mask[seeds[0]] = True

    for _ in range(n_parts - 1):
        dist = torch.full((N,), float('inf'), device=device)
        frontier = selected_mask.clone().float()
        current_dist = 0

        while frontier.sum() > 0:
            newly_visited = (dist == float('inf')) & (frontier > 0)
            dist = torch.where(newly_visited, current_dist, dist)
            frontier = torch.sparse.mm(adj, frontier.unsqueeze(1)).squeeze() > 0
            frontier = frontier & (dist == float('inf'))
            frontier = frontier.float()
            current_dist += 1

        farthest = torch.argmax(dist).item()
        seeds.append(farthest)
        selected_mask[farthest] = True

    return torch.tensor(seeds, device=device)

def region_growing_partition(edge, n_parts, num_elements, device="cuda:2"):
    device = torch.device(device)
    adj = build_adjacency_matrix(edge, num_elements, device)
    seeds = pick_distant_seeds(adj, n_parts)

    labels = torch.full((num_elements,), -1, dtype=torch.long, device=device)
    labels[seeds] = torch.arange(n_parts, device=device)

    frontier = torch.zeros((n_parts, num_elements), dtype=torch.bool, device=device)
    frontier[torch.arange(n_parts), seeds] = True

    while (labels == -1).any():
        expanded = torch.sparse.mm(adj, frontier.float().T).T > 0
        expanded = expanded & (labels == -1).unsqueeze(0)
        new_indices = expanded.nonzero(as_tuple=False)
        labels[new_indices[:, 1]] = new_indices[:, 0]
        frontier = expanded

    groups = [torch.nonzero(labels == i, as_tuple=True)[0] for i in range(n_parts)]
    return groups, seeds

def build_sparse_K_local(K, elements, element_indices, device="cuda:2"):
    device = torch.device(device)
    K = K.to(device)
    elements = elements.to(device)
    K_part = K[element_indices]
    elems = elements[element_indices]

    global_nodes = torch.unique(elems)
    global_to_local = {int(n): i for i, n in enumerate(global_nodes.tolist())}
    flat_elems = elems.view(-1).tolist()
    local_node_ids = torch.tensor([global_to_local[n] for n in flat_elems], device=device).view(elems.shape)
    elems_local = local_node_ids

    node_idx = elems_local.unsqueeze(-1).repeat(1, 1, 3)
    dof_idx = node_idx * 3 + torch.arange(3, device=device).view(1, 1, 3)
    dof_idx = dof_idx.view(len(elems_local), -1)

    row_idx = dof_idx.unsqueeze(2).repeat(1, 1, 12).view(-1)
    col_idx = dof_idx.unsqueeze(1).repeat(1, 12, 1).view(-1)
    values = K_part.view(-1)

    N_nodes = elems_local.max().item() + 1
    N_dof = N_nodes * 3

    K_local = torch.sparse_coo_tensor(
        indices=torch.stack([row_idx, col_idx]),
        values=values,
        size=(N_dof, N_dof),
        device=device
    )

    return K_local, global_nodes

def partition_and_build_sparse_K(K, elements, edge, n_parts, device="cuda:2"):
    device = torch.device(device)
    groups, seeds = region_growing_partition(edge, n_parts, K.shape[0], device=device)
    K_parts = []
    node_maps = []
    for g in groups:
        K_local, global_nodes = build_sparse_K_local(K, elements, g, device=device)
        K_parts.append(K_local)
        node_maps.append(global_nodes)
    return K_parts, node_maps, groups, seeds

In [11]:
K_parts, node_maps, element_groups, seeds = partition_and_build_sparse_K(K, elements, edge, n_parts=20, device="cuda:2")

In [12]:
print("Memory allocated on GPU for partitioned sparse K: ", f"{torch.cuda.memory_allocated(K.device) / 1024**3:.4f} GB")

Memory allocated on GPU for partitioned sparse K:  4.4562 GB


In [13]:
def invert_K_parts(K_parts):
    K_inverses = []
    for K in K_parts:
        K_dense = K.to_dense()
        K_inv = torch.linalg.inv(K_dense)
        K_inverses.append(K_inv)
    return K_inverses

K_inverses = invert_K_parts(K_parts)
print("Memory allocated on GPU for partitioned inverted sparse K: ", f"{torch.cuda.memory_allocated(K.device) / 1024**3:.4f} GB")

Memory allocated on GPU for partitioned inverted sparse K:  39.8502 GB


In [73]:
def build_ordered_subdomain_map(node_maps):
    node_groups = {}
    for sd_idx, nds in enumerate([nm.cpu().tolist() for nm in node_maps]):
        for nd in nds:
            if nd not in node_groups:
                node_groups[nd] = set()
            node_groups[nd].add(sd_idx)
    group_to_nodes = defaultdict(list)
    for nd, sds in node_groups.items():
        if len(sds)>1:
            group_to_nodes[tuple(sorted(sds))].append(nd)
    return dict(group_to_nodes)

group_to_nodes = build_ordered_subdomain_map(node_maps)
print(len(group_to_nodes.keys()))

105


In [80]:
free_variables = []
for k, v in group_to_nodes.items():
    free_variables.extend([[0,0,0]]*((len(k)-1)*len(v)))
free_varibles = torch.tensor(free_variables, device=device)
print(free_varibles.shape)

torch.Size([11369, 3])


In [ ]:
def make_sub_domain_forces(free_vars, group_to_nodes, rbe2, F, n_sub, device="cuda"):
    s = [F.clone().to(device) for i in range(n_sub)]
    off = 0
    for k,v in group_to_nodes.items():
        m = len(k) - 1
        c = m * len(v)
        chunk = free_vars[off:off + c]
        off += c
        idx = 0
        sd = sorted(k)
        for nd in v:
            if nd in rbe2:
                idx += m
                continue
            if m == 1:
                s[sd[0]][nd] += chunk[idx]
                s[sd[1]][nd] -= chunk[idx]
                idx += 1
            else:
                s[sd[0]][nd] += chunk[idx]
                for j in range(1,m):
                    s[sd[j]][nd] += chunk[idx+j] - chunk[idx+j-1]
                s[sd[m]][nd] -= chunk[idx+m-1]
                idx += m
    return s